# Consistent Hashing

In [1]:
import hashlib
from bisect import bisect, bisect_left
from typing import Union

import matplotlib.pyplot as plt

In [2]:
def hash_fn(data: Union[str, int]) -> int:
    """hash_fn creates an integer equivalent of a SHA256 hash

    data could be a string or an integer
    """
    hsh = hashlib.sha256()

    # if data is str then encode it before converting it to bytes
    if type(data) in [str]:
        data = data.encode('utf-8')

    # converting data into bytes and passing it to hash function
    hsh.update(bytes(data))

    # converting the HEX digest into equivalent integer value
    return int(hsh.hexdigest(), 16)

In [3]:
class HashRing:
    """HashRing represents Consistent HashRing. One instance of this
    class will be one instance of Consistent HashRing. The class exposes
    all the functions required to interact with the ring.
    """

    def __init__(self):
        self.count = 0
        self.keys = []
        self.nodes = []
        self.hash_space = 100

    def _generate_key(self, data):
        """returns hash key modulo hash_space because of which it represents
        the location in the flattened hash ring where the data should reside.
        """
        return hash_fn(data) % self.hash_space

    def add_node(self, node_id: Union[str, int]) -> None:
        """add_node function adds a new node to the ring
        """
        # Error handle when hash space is full.
        if self.count == self.hash_space:
            raise Exception("hash space is full")

        self.count += 1
        key = self._generate_key(node_id)

        # find the index where the key should be inserted in the keys store
        index = bisect(self.keys, key)

        # insert the node_id and the key at the same `index` location.
        # this insertion will keep nodes and keys sorted w.r.t keys.
        self.nodes.insert(index, node_id)
        self.keys.insert(index, key)

    def remove_node(self, node_id: Union[str, int]) -> None:
        if self.count == 0:
            raise Exception("hash space is empty")

        key = hash_fn(node_id) % self.hash_space

        index = bisect_left(self.keys, key)
        try:
            self.keys.remove(key)
            self.nodes.pop(index)
            self.count -= 1
        except ValueError:
            pass

    def plot(self) -> None:
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 3, 1])
        ax.axes.get_yaxis().set_visible(False)
        plt.ylim(top=2)
        data = [0] * self.hash_space
        for k in self.keys:
            data[k] = 1
        barlist = ax.bar(range(self.hash_space), data)

        # barlist[self.keys[0]].set_color('r')
        plt.show()

In [4]:
hr = HashRing()